In [ ]:
# Fine-Tuning PuoBERTa with 5-Fold CV on 80% Train Set + Final Test on 20% Holdout
# Dataset: 477 Offensive, 500 Non-Offensive Samples

"""
Fine-tuning PuoBERTa with 5-fold cross-validation on an 80% training split,
followed by final evaluation on a 20% holdout test set.

Dataset (after preprocessing and splitting):
- 477 Offensive samples
- 500 Non-offensive samples

This script assumes you already have:
- train.csv  (80% of data)
- test.csv   (20% holdout)

both stored in a local ./data/ directory.
"""
!pip install transformers datasets torch scikit-learn

import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, matthews_corrcoef
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, RobertaForSequenceClassification, RobertaTokenizer
from datasets import Dataset
from torch.nn import CrossEntropyLoss

# Directory where preprocessed CSV files are stored.
# For public code, we assume a local ./data/ folder in the repo.
DATA_DIR = Path("data")

# Load training (80%) and testing (20%) CSV files
train_df = pd.read_csv('DATA_DIR/train.csv')
test_df = pd.read_csv('DATA_DIR/test.csv')
train_df['TEXT'] = train_df['TEXT'].astype(str)
test_df['TEXT'] = test_df['TEXT'].astype(str)

# Load tokenizer and add trigger tokens
tokenizer = RobertaTokenizer.from_pretrained('dsfsi/PuoBERTa')
# tokenizer.add_special_tokens({'additional_special_tokens': ['<TRIGGER>', '</TRIGGER>']})

def tokenize_batch(batch):
    return tokenizer(batch['TEXT'], padding='max_length', truncation=True, max_length=128)

# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    return {
        'accuracy': accuracy_score(labels, preds),
        'mcc': matthews_corrcoef(labels, preds),
        'roc_auc': roc_auc_score(labels, torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()),
        'f1_macro': f1.mean(),
        'recall_1': recall[1]
    }

# Define custom loss function for class imbalance
def compute_weighted_loss(model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    outputs = model(**inputs)
    loss_fct = CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]).to(model.device))
    loss = loss_fct(outputs.logits.view(-1, 2), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

# Training arguments
training_args = TrainingArguments(
    output_dir='DATA_DIR/PuoBERTa-finetuned',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='no',
    eval_strategy='epoch',
    metric_for_best_model='recall_1',
    greater_is_better=True,
    load_best_model_at_end=False
)

# Apply 5-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []
best_models = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_df['TEXT'], train_df['TARGET'])):
    print(f'\n=== Fold {fold + 1}/5 ===')
    train_fold = train_df.iloc[train_idx]
    val_fold = train_df.iloc[val_idx]

    train_dataset = Dataset.from_pandas(train_fold).map(tokenize_batch, batched=True)
    val_dataset = Dataset.from_pandas(val_fold).map(tokenize_batch, batched=True)
    train_dataset = train_dataset.rename_column('TARGET', 'label')
    val_dataset = val_dataset.rename_column('TARGET', 'label')

    # Convert 'label' column to integers using a mapping
    label_mapping = {'Non-offensive': 0, 'Offensive': 1}
    train_dataset = train_dataset.map(lambda examples: {'label': [label_mapping[x] for x in examples['label']]}, batched=True)
    val_dataset = val_dataset.map(lambda examples: {'label': [label_mapping[x] for x in examples['label']]}, batched=True)

    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # ---------------------------------------------------------------------
    # Configuration
    # ---------------------------------------------------------------------
    # Base model used for fine-tuning. Replace this with your fine-tuning
    # model on Hugging Face, e.g. "dsfsi/PuoBERTa".

    model = RobertaForSequenceClassification.from_pretrained('dsfsi/PuoBERTa', num_labels=2)

    # Optional: if your experiments use explicit trigger markers, document it:
    # SPECIAL_TOKENS = {"additional_special_tokens": ["<TRIGGER>", "</TRIGGER>"]}
    # NOTE: When adding special tokens you must also resize the model embeddings:
    # model.resize_token_embeddings(len(tokenizer))

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.loss_function = compute_weighted_loss
    trainer.train()
    metrics = trainer.evaluate()
    fold_results.append(metrics)
    best_models.append(model)
    print(metrics)
# Final evaluation on the held-out 20% test set
best_fold_index = np.argmax([r['eval_recall_1'] for r in fold_results])
best_model = best_models[best_fold_index]
print(f'\nBest Fold: Fold {best_fold_index + 1}')

test_dataset = Dataset.from_pandas(test_df).map(tokenize_batch, batched=True)
test_dataset = test_dataset.rename_column('TARGET', 'label') # Changed 'TARGET' to 'TARGET LABEL'

# Convert 'label' column to integers using a mapping before setting the format
label_mapping = {'Non-offensive': 0, 'Offensive': 1}
test_dataset = test_dataset.map(lambda examples: {'label': [label_mapping[x] for x in examples['label']]}, batched=True)

test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

test_trainer = Trainer(
    model=best_model,
    args=TrainingArguments(output_dir='./tmp-test', per_device_eval_batch_size=64),
    compute_metrics=compute_metrics
)
test_metrics = test_trainer.evaluate(test_dataset)
print('\n=== Test Set Metrics ===')
print(test_metrics)

# Save best model and tokenizer to Google Drive
save_dir = 'DATA_DIR/PuoBERTa-setswana-offensive-v11'
best_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Best performing model saved to: {save_dir}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/877k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/523k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,Mcc,Roc Auc,F1 Macro,Recall 1
1,0.632300,0.510782,0.807692,0.620505,0.916502,0.806899,0.743590
2,0.429600,0.353000,0.852564,0.706581,0.935733,0.852412,0.820513
3,0.303700,0.313025,0.871795,0.744569,0.938199,0.871711,0.897436
4,0.223700,0.314455,0.865385,0.733730,0.937541,0.865113,0.910256
5,0.206000,0.309591,0.871795,0.743834,0.938856,0.871774,0.884615


{'eval_loss': 0.30959075689315796, 'eval_accuracy': 0.8717948717948718, 'eval_mcc': 0.7438343052617367, 'eval_roc_auc': 0.9388560157790927, 'eval_f1_macro': 0.8717737958244287, 'eval_recall_1': 0.8846153846153846, 'eval_runtime': 0.4731, 'eval_samples_per_second': 329.722, 'eval_steps_per_second': 6.341, 'epoch': 5.0}

Best Fold: Fold 5


Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]


=== Test Set Metrics ===
{'eval_loss': 0.3381184935569763, 'eval_model_preparation_time': 0.0027, 'eval_accuracy': 0.8673469387755102, 'eval_mcc': 0.7326205066429489, 'eval_roc_auc': 0.9288259958071279, 'eval_f1_macro': 0.8662186285834296, 'eval_recall_1': 0.8444444444444444, 'eval_runtime': 0.5897, 'eval_samples_per_second': 332.398, 'eval_steps_per_second': 6.784}
Best performing model saved to: /content/drive/My Drive/Colab Notebooks/PuoBERTa-best-fold-model-80-20
